# 기본 구조 작성하기

### 아래의 그림 구조 만들어보기

1. 사용자 요청
2. 쿼리 3가지로 증강
3. gpt와 claude 병렬 요청
4. 결과 종합
5. 결과 판단
6. rewrite query and 2번 반복
7. 종료

!["structure"](/home/ansgyqja/AI_application/images/langgraph-building-graphs.png)

In [29]:
from typing import Annotated,Literal,List
from langchain_core.documents import Document
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langchain_teddynote.tools.tavily import TavilySearch
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage, RemoveMessage, HumanMessage

In [ ]:


# State 정의
class State(TypedDict):
    context: Annotated[List[Document], add_messages]
    answer: Annotated[List[Document], add_messages]
    question: Annotated[str, "user question"]
    binary_score: Annotated[list, add_messages]

def retrieve(state: State) -> State:
    # retrieve: 검색
    documents = "검색된 문서"
    return {"context": documents}
    
def gpt_generate(state: State) -> State:
    # gpt_request: 검색
    documents = "gpt 답변"
    return {"answer": documents}

def claude_generate(state: State) -> State:
    # claude_request: 검색
    documents = "claude 답변"
    return {"answer": documents}

def revalance_check(state:State) -> State:
    # gpt or claude response 검색
    return State({'binary_score':"yes"})

def sum_answer(state:State)->State:
    # sum_answer 검색
    answer = "종합된 답변"
    return State(answer=answer)

def decision(state: State) -> Literal["rewrite_query",END]:
    # 의사결정
    decision = "결정"
    # 로직을 추가할 수 가 있고요.
    
    if state["binary_score"] == "no" :
        return "rewrite_query"
    return END
    
def rewrite_query(state: State)->State:
    question = "새로운 질문"
    return State({'question':question})

In [31]:


state_graph = StateGraph(State)

state_graph.add_node('retrieve',retrieve)
state_graph.add_node('gpt_generate',gpt_generate)
state_graph.add_node('claude_generate',claude_generate)
state_graph.add_node('gpt_revalance_check',revalance_check)
state_graph.add_node('claude_revalance_check',revalance_check)
state_graph.add_node('sum_answer',sum_answer)
state_graph.add_node('rewrite_query',rewrite_query)

state_graph.add_edge(START,'retrieve')

state_graph.add_edge('retrieve','gpt_generate')
state_graph.add_edge('gpt_generate','gpt_revalance_check')
state_graph.add_edge('gpt_revalance_check','sum_answer')

state_graph.add_edge('retrieve','claude_generate')
state_graph.add_edge('claude_generate','claude_revalance_check')
state_graph.add_edge('claude_revalance_check','sum_answer')

state_graph.add_conditional_edges(
    source='sum_answer',
    path=decision
)
state_graph.add_edge('rewrite_query','retrieve')


In [32]:
# 메모리 저장소 생성
memory = MemorySaver()
graph = state_graph.compile(checkpointer=memory)
mermaid_code = graph.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	retrieve(retrieve)
	gpt_generate(gpt_generate)
	claude_generate(claude_generate)
	gpt_revalance_check(gpt_revalance_check)
	claude_revalance_check(claude_revalance_check)
	sum_answer(sum_answer)
	rewrite_query(rewrite_query)
	__end__([<p>__end__</p>]):::last
	__start__ --> retrieve;
	claude_generate --> claude_revalance_check;
	claude_revalance_check --> sum_answer;
	gpt_generate --> gpt_revalance_check;
	gpt_revalance_check --> sum_answer;
	retrieve --> claude_generate;
	retrieve --> gpt_generate;
	rewrite_query --> retrieve;
	sum_answer -.-> __end__;
	sum_answer -.-> rewrite_query;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [33]:
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(
    recursion_limit=6,  # 최대 10개의 노드까지 방문. 그 이상은 RecursionError 발생
    configurable={"thread_id": "3"},  # 스레드 ID 설정
)

In [34]:
query = "안녕"
for event in graph.stream({"question":query},config=config,stream_mode="values"):
    print("**"*30)
    print(event)

************************************************************
{'context': [], 'answer': [], 'question': '안녕', 'binary_score': []}
************************************************************
{'context': [HumanMessage(content='검색된 문서', additional_kwargs={}, response_metadata={}, id='d73878a5-3f2f-4b9f-85a1-f3676f4979b2')], 'answer': [], 'question': '안녕', 'binary_score': []}
************************************************************
{'context': [HumanMessage(content='검색된 문서', additional_kwargs={}, response_metadata={}, id='d73878a5-3f2f-4b9f-85a1-f3676f4979b2')], 'answer': [HumanMessage(content='claude 답변', additional_kwargs={}, response_metadata={}, id='49abd9de-a4b6-48ad-a036-349659bd7a33'), HumanMessage(content='gpt 답변', additional_kwargs={}, response_metadata={}, id='9749646e-e197-437d-bd45-85ee34f18e1c')], 'question': '안녕', 'binary_score': []}
************************************************************
{'context': [HumanMessage(content='검색된 문서', additional_kwargs={}, response_meta